## Extraction et Prédiction à partir d'une Vidéo

Ce notebook contient le code permettant d'obtenir des prédictions des lettres de l'alphabet du langage des signes américain (ASL) à partir d'une vidéo.

### Méthode d'extraction des images
Pour réaliser les prédictions, nous avons choisi d'extraire **2 images par seconde (fps)** à partir de la vidéo.

### Stratégie pour améliorer les prédictions
Les modèles générés par notre réseau de neurones, n'étant pas très fiable fiable et de précis. En particulier, une même image peut parfois être associée à des lettres différentes selon les prédictions du modèle. Nous avons décidé pour y remédier de:
1. Passer **chaque image 5 fois** dans le même modèle.
2. Attribuer comme prédiction finale la lettre ayant la **plus grande occurrence parmi les trois résultats**.

### Gestion des répétitions dans les prédictions
Un autre problème est apparu lors du traitement : en extrayant 2 images par seconde, il arrive que plusieurs images consécutives soient associées à la même lettre, même lorsque ce n'est pas le cas dans la vidéo originale. Par exemple, si une personne énonce "Emma", le résultat brut pourrait inclure des lettres répétées comme "Eemmaaa". Puisque nous ne pouvons pas prédire à quelle vitesse la personne va signer dans la vidéo.

Pour éviter ces répétitions et améliorer la lisibilité, nous avons choisi de conserver uniquement les lettres **différentes** dans la séquence finale. Ainsi :
- Si une lettre est détectée plusieurs fois de suite, elle est comptée une seule fois.
- Par exemple, pour le prénom "Emma", le résultat serait "Ema".

Nous avons fait ce choix en nous basant sur le fait que **la compréhension du message est prioritaire par rapport à l'orthographe exacte**.



# Imports de librairies

In [ ]:
!pip install mediapipe

In [2]:
import os
import cv2
import shutil
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageOps
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Import du modèle

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'gpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [44]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'vide']
num_classes = len(class_names) # = 27
label_to_class = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'vide'}

Les 2 codes suivant est a adapté suivant le modèle que l'on télécharge

In [21]:
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

In [4]:
class ImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5), #output_shape=(batch_size, 16, 196, 196)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #output_shape=(batch_size, 16, 98, 98)
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5), #output_shape=(batch_size, 32, 94, 94)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #output_shape=(batch_size, 32, 47, 47)
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5), #output_shape=(batch_size, 64, 43, 43)
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5), #output_shape=(batch_size, 128, 39, 39)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #output_shape=(batch_size, 128, 19, 19)
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(128 * 19 * 19, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        logits = self.convolution(x)
        return logits

Chargement modèle au format .pth

In [ ]:
filename = 'full_model.pth'

checkpoint = torch.load(filename)
model = ImageClassifier(num_classes=num_classes).to(device)  # Dépend du modèle utilisé
model.load_state_dict(checkpoint['model_state_dict'])
optimizer = torch.optim.SGD(model.parameters())
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# Découpage de la vidéo

In [6]:
def deleteDir(path):
  """
  Cette fonction permet de supprimer un dossier et son contenu.

  Arguments:
  - path: chemin du dossier à supprimer
  """
  if os.path.exists(path):
    shutil.rmtree(path)

In [12]:
def decoupeVideo(video_path, output_folder):
  """
  Cette fonction permet de découper une vidéo en images (2 images par secondes).
  Elle découpe la vidéo en images et les enregistres dans le dossier 'output_folder'.

  Arguments:
  - video_path: chemin vers la vidéo à découper
  - output_folder: dossier où les images seront enregistrées.
  """

  # Si le dossier d'extraction existe déjà --> suppression
  # Pour supprimer des images potentiellement
  # déjà découpées d'une vidéo précédente
  deleteDir(output_folder)
  os.makedirs(output_folder)  # création du dossier d'extraction

  video = cv2.VideoCapture(video_path) # Ouverture vidéo

  # Vérification de l'ouverture de la vidéo
  if not video.isOpened():
      print("Erreur ouverture vidéo (verifiez si le chemin de la video est ok)")
  else:

      fps = video.get(cv2.CAP_PROP_FPS)  # Frames Par Seconde

      frame_cpt = 0  # Compteur pour nommer les images extraites
      FRAME_INTERVAL = int(fps // 2) # On extrait 2 images par secondes

      while True:  # Tant qu'on a pas lu toute la vidéo
          # Lecture frame 1
          success, frame = video.read()
          if not success:
              break  # Fin de la vidéo

          # Si frame lue, on saute les frames nécessaires pour obtenir 2 fps
          for i in range(FRAME_INTERVAL - 1):
              success, _ = video.read()
              if not success:
                  break  # Fin de la vidéo

          # Sauvegarde de l'image
          if success:
              frame_filename = os.path.join(output_folder, f"{os.path.basename(video_path)}_{frame_cpt:04d}.jpg")
              cv2.imwrite(frame_filename, frame)
              frame_cpt += 1

      video.release()
      print(f"Découpage video {video_path} terminé. {frame_cpt} images ont été extraites.")

In [13]:
video_path = "asl.mp4"  # Entrez le chemin de la vidéo ici

In [ ]:
output_folder = "output_images"
decoupeVideo(video_path, output_folder)

# Traitement des images extraites

Les fonctions suivantes sont déjà présente dans le notebook précédent mais étant nécessaire à la prédiction nous les avons réécrites ici.

In [15]:
"""
!!! OpenCV charge les images au format BGR (Bleu, Vert, Rouge)
Matplotlib attend les images au format RGB (Rouge, Vert, Bleu) !!!
"""

# La fonction est déjà présente en haut du notebook
def zoomMain(filename):
    """
    Cette fonction permet de zoomer sur la main d'une image.
    Ne renvoie pas d'erreur si une image n'est pas détectée.
    De plus, le zoom sur la main génére une image qui écrase celle d'origine.

    Arguments:
    - filename: chemin vers l'image à zoomer
    """

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    # Chargement image
    image = cv2.imread(filename)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversion bgr --> rgb

    # Détection de la main
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Récupérer les coordonnées des landmarks (points importants de la main)
            h, w, _ = image.shape
            landmarks = [(int(pt.x * w), int(pt.y * h)) for pt in hand_landmarks.landmark]

            # Calculer la bounding box
            x_min = min([pt[0] for pt in landmarks])
            x_max = max([pt[0] for pt in landmarks])
            y_min = min([pt[1] for pt in landmarks])
            y_max = max([pt[1] for pt in landmarks])

            # Ajout marge autour de la main
            marge = 300  # On a décidé de mettre 300 car c'était ce qui nous paraissait le mieux après avoir fait divers test sur des images
            x_min = max(0, x_min - marge)
            y_min = max(0, y_min - marge)
            x_max = min(w, x_max + marge)
            y_max = min(h, y_max + marge)

            # Extraction et zoom sur la main
            hand_image = image[y_min:y_max, x_min:x_max]
            hand_image_rgb = cv2.cvtColor(hand_image, cv2.COLOR_BGR2RGB)  # Conversion BGR -> RGB

            # Enregistre la main zoomée
            cv2.imwrite(filename, cv2.cvtColor(hand_image_rgb, cv2.COLOR_RGB2BGR))

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Fermeture Mediapipe
    hands.close()

In [16]:
# Nous avons choisit d'implémenter cette fonction car celle fournit
# par pytorch ne conservait les proportions d'images
def resize_properly(image_path, output_path="*", target_size=(300, 300)):
    """
    Redimensionne une image proprement en gardant le bon ratio de l'image d'origine
    Et a fait correspondre à une taille cible (target_size)
    Ainsi les points important de l'image cible seront toujours là dans l'image redimensionnée

    image_path: Chemin de l'image source
    output_path: Chemin pour enregistrer l'image redimensionnée
    target_size: Tuple (largeur, hauteur) de la taille cible
    """

    # Si "*" alors on écrase l'image d'origine
    if output_path == "*":
      output_path= image_path

    target_largeur, target_hauteur = target_size

    # Ouverture l'image
    image = Image.open(image_path)
    image = ImageOps.exif_transpose(image)  # Garde l'orientation de l'image d'origine (verticale ou horizontale)
    original_largeur, original_hauteur = image.size
    ratio = original_largeur / original_hauteur

    # Calcule nouvelle taille de l'image en conservant le ratio
    if ratio > target_largeur / target_hauteur:
        # Si image cible plus large que haute
        # --> On ajuste la hauteur
        new_largeur = target_largeur
        new_hauteur = int(target_largeur / ratio)  # un nb de pixel est un entier
    else:
        # Si image plus haute que large
        # --> On ajuste la largeur
        new_hauteur = target_hauteur
        new_largeur = int(target_hauteur * ratio)

    # Redimensionne l'image avec le bon ratio
    resized_image = image.resize((new_largeur, new_hauteur))

    # Padding en noir de l'image pour avoir la taille cible
    new_image = Image.new("RGB", target_size, (0, 0, 0))  # (0, 0, 0) rgb --> noir
    new_image.paste(resized_image,
                    ((target_largeur - new_largeur) // 2, (target_hauteur - new_hauteur) // 2))
                    # copie de l'image redimensionnée dans le cadre noir de bonne dimension
                    # en la centrant au milieu du fond noir

    # sauvergarde
    new_image.save(output_path)

In [ ]:
dir = 'output_images'

for image in os.listdir(dir):
    image_path = os.path.join(dir, image)

    if os.path.isfile(image_path):
        zoomMain(image_path)
        resize_properly(image_path, "*", (200,200))
print("Images modifiées")

# Prédiction sur vidéo

In [60]:
from collections import Counter

def word_prediction():
  val_images = []

  for dirname, _, filenames in os.walk('output_images'):
      for filename in filenames:

          path = os.path.join(dirname, filename)
          img = Image.open(path)
          img_tensor = transform(img).unsqueeze(0).to(device)

          pred_labels = []
          model.eval()
          with torch.no_grad():
              for _ in range(5):  # fait 5 prédictions en garde celle qui à la plus grande occurence
                  output = model(img_tensor)  # Prédiction
                  pred_labels.append(output.argmax(1).item())  # stockage des labels prédits 5 fois

          # Compte les occurrences des prédictions
          cpt = Counter(pred_labels)
          val_most_frequent = cpt.most_common(1)[0][0]  # Classe prédite avec la + grande occurrence

          val_images.append(val_most_frequent)

  return val_images

Le code suivant permet d'afficher la phrase ou le mot prédit par le modèle:

In [61]:
val_images = word_prediction()

next_pred = ""
print("La phrase ou le mot de la vidéo est:")
for val in val_images:
  pred = label_to_class[val]

  if pred != next_pred:  # On empeche que le mot comporte 2 meme lettre à la suite
    print(pred, end="")
    next_pred = pred

La phrase ou le mot de la vidéo est:
BTITITXITIXBTITETUITITYITIBIVTUEFITBXVTITXTXTITOITBTXIXBTVTLQITBUIT